In [12]:
#id-2 - [1,0], id_7 - [0,1]
import tensorflow as tf 

In [19]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import random

In [49]:
data_x, data_y = [], []
img_length = 500*500*3

In [50]:
folder = 'pixiv/id_2'
for img_file in os.listdir(folder):
    img=np.array(Image.open(folder+'/'+img_file))
#     img=np.array(Image.open('/'.join([folder, img_file])))
#     img = np.reshape(img, 500*500*3)
#     img = np.reshape(img, len(img)*len(img[0])*len(img[0][0]))
#     print(len(img))
#     print(img)
    data_x.append(img/255.0)
    data_y.append([1,0])
print(len(data_x))
print(len(data_y))

200
200


In [51]:
folder = 'pixiv/id_7'
for img_file in os.listdir(folder):
    img=np.array(Image.open(folder+'/'+img_file))
#     img=np.array(Image.open('/'.join([folder, img_file])))
#     img = np.reshape(img, 500*500*3)
#     img = np.reshape(img, len(img)*len(img[0])*len(img[0][0]))
#     print(len(img))
#     print(img)
    data_x.append(img/255.0)
    data_y.append([0,1])
print(len(data_x))
print(len(data_y))

400
400


In [52]:
data_x = np.array(data_x, dtype='float32')
data_y = np.array(data_y, dtype='float32')

print(data_x[:10])
print(data_y[:10])

[[[[ 0.36470589  0.36078432  0.58823532]
   [ 0.32549021  0.32156864  0.54901963]
   [ 0.3137255   0.30980393  0.53725493]
   ..., 
   [ 0.31764707  0.32156864  0.50196081]
   [ 0.32941177  0.33333334  0.51372552]
   [ 0.36862746  0.37254903  0.5529412 ]]

  [[ 0.29803923  0.29411766  0.52941179]
   [ 0.27450982  0.27058825  0.49803922]
   [ 0.27843139  0.27450982  0.50196081]
   ..., 
   [ 0.25490198  0.25882354  0.43921569]
   [ 0.25882354  0.26274511  0.44313726]
   [ 0.29411766  0.29803923  0.47843137]]

  [[ 0.3137255   0.30980393  0.54509807]
   [ 0.27450982  0.27058825  0.50588238]
   [ 0.26274511  0.25882354  0.48627451]
   ..., 
   [ 0.25490198  0.25882354  0.44705883]
   [ 0.26666668  0.27058825  0.45882353]
   [ 0.30588236  0.30980393  0.49019608]]

  ..., 
  [[ 0.24705882  0.23921569  0.38431373]
   [ 0.20784314  0.2         0.34509805]
   [ 0.2         0.19215687  0.34509805]
   ..., 
   [ 0.22352941  0.21176471  0.39607844]
   [ 0.23137255  0.21960784  0.40392157]
   [ 0.

In [53]:
order = [i for i in range(len(data_x))]
random.shuffle(order)

print(len(order))
n=int(len(data_x)*9/10)
print(n)

train_x = [data_x[i] for i in order[0:n]]
train_y = [data_y[i] for i in order[0:n]]

test_x = [data_x[i] for i in order[n:]]
test_y = [data_y[i] for i in order[n:]]

print(len(train_x))
print(len(test_y))

400
360
360
40


In [62]:
graph = tf.Graph()
with graph.as_default(), tf.device('/gpu:0'):
    labels = tf.placeholder('float', [None, 2])
    inputs = tf.placeholder('float', [None, 500, 500, 3])   
#     inputs = tf.placeholder('float', [None, 500*500*3])                        

    def weight_variable(shape): #定义一个函数，用于初始化所有的权值 W
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial)

    def bias_variable(shape): #定义一个函数，用于初始化所有的偏置项 b
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial)

    def conv2d(x, W): #定义一个函数，用于构建卷积层
        return tf.nn.conv2d(x, W, strides=[1, 2, 2, 1], padding='VALID')

    def max_pool(x): #定义一个函数，用于构建池化层
        return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')

    #构建网络
#     x_image = tf.reshape(inputs, [-1,500,500,3])         #转换输入数据shape,以便于用于网络中
    W_conv1 = weight_variable([5, 5, 3, 32])      
    b_conv1 = bias_variable([32])       
    h_conv1 = tf.nn.relu(conv2d(inputs, W_conv1) + b_conv1)     #第一个卷积层
    h_pool1 = max_pool(h_conv1)                                  #第一个池化层
    print(h_pool1.shape)#(?, 124, 124, 32)

    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)      #第二个卷积层
    h_pool2 = max_pool(h_conv2)                                   #第二个池化层
    print(h_pool2.shape)#(?, 30, 30, 64)

    # 第三层 是个全连接层,输入维数125*125*64, 输出维数为1024
    W_fc1 = weight_variable([30 * 30 * 64, 1024])
    b_fc1 = bias_variable([1024])
    h_pool2_flat = tf.reshape(h_pool2, [-1, 30*30*64])              #reshape成向量
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)    #第一个全连接层

    keep_prob = tf.placeholder("float") 
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)                  #dropout层

    W_fc2 = weight_variable([1024, 2])
    b_fc2 = bias_variable([2])
#     y_predict=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)   #softmax层
    y_predict=tf.matmul(h_fc1_drop, W_fc2) + b_fc2

#     cross_entropy = -tf.reduce_sum(labels*tf.log(y_predict))     #交叉熵
#     train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)    #梯度下降法
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_predict, labels = labels))
    optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)    
    
    correct_prediction = tf.equal(tf.argmax(y_predict,1), tf.argmax(labels,1))    
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))    #精确度计算

(?, 124, 124, 32)
(?, 30, 30, 64)


In [66]:
batch_start = 0
batch_size = 50
training_steps = 501

# print(test_y[0])

with tf.Session(graph=graph) as session:
#     saver.restore(session, tf.train.latest_checkpoint('lstm_check'))
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(training_steps): 
        if(batch_start+batch_size>=len(train_x)):
            order = [i for i in range(len(train_x))]
            random.shuffle(order)
            train_x = [train_x[i] for i in order]
            train_y = [train_y[i] for i in order]
            batch_start = 0        
        feed_x = train_x[batch_start:batch_start+batch_size]
        feed_y = train_y[batch_start:batch_start+batch_size]
#         print(feed_x.shape)
#         print(feed_y.shape)
        feed_dict = {inputs: feed_x, labels: feed_y, keep_prob:1.0}
        _, l, predictions, accur = session.run([optimizer, loss, labels, accuracy], feed_dict = feed_dict)
        if (step % 50 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('accuarcy is ', accur)
#             saver.save(session, 'lstm_check/my-model-1.ckpt', global_step=step+1000)
    test_loss, test_logits, test_accur = session.run([loss,labels,accuracy], feed_dict={inputs: test_x, labels: test_y, keep_prob:1.0})

Initialized
Loss at step 0: 5.702176
accuarcy is  0.48
Loss at step 50: 0.776365
accuarcy is  0.58
Loss at step 100: 0.138931
accuarcy is  0.98
Loss at step 150: 0.070234
accuarcy is  0.98
Loss at step 200: 0.038352
accuarcy is  1.0
Loss at step 250: 0.023218
accuarcy is  1.0
Loss at step 300: 0.015201
accuarcy is  1.0
Loss at step 350: 0.010549
accuarcy is  1.0
Loss at step 400: 0.007656
accuarcy is  1.0
Loss at step 450: 0.005675
accuarcy is  1.0
Loss at step 500: 0.004378
accuarcy is  1.0


In [68]:
print('test loss, ', test_loss, ' test accur is ', test_accur)

test loss,  0.564804  test accur is  0.825
